In [ ]:
# First, clear any existing large DataFrames from memory
try:
    del player_statistics_df
    import gc
    gc.collect()
    print("Memory cleared from previous DataFrame")
except NameError:
    print("No existing DataFrame found in memory")

# Remove old csv files if there are any
try:
    os.remove("player-statistics-new.csv")
    os.remove("player-statistics.csv")
except:
    pass


import pandas as pd
import unicodedata
import re
import duckdb
import wget
import os

# Download files
print("Downloading files...")
filename = 'player-statistics.csv'
url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
wget.download(url)
print(f"\nDownloaded {filename}")

# Define the output file
output_file = 'player-statistics-new.csv'

# Process and save in chunks
chunk_size = 100000
first_chunk = True
processed_rows = 0

print(f"Processing data in chunks of {chunk_size} rows...")

for chunk_num, chunk in enumerate(pd.read_csv('player-statistics.csv', chunksize=chunk_size, low_memory=False)):

    if chunk_num > 0:
        break

    # Register current chunk with DuckDB
    duckdb.register('player_statistics_chunk', chunk)

    query = """
    SELECT *

/*    firstName
    ,lastName
    ,full_name
    ,player_id
    ,gameId
    ,gameDate AS DATE
    ,playerteamCity
    ,playerteamName
    ,opponentteamCity
    ,opponentteamName
    ,gameType
    ,gameLabel
    ,gameSubLabel
    ,seriesGameNumber
    ,win
    ,home
    ,numMinutes
    ,points
    ,assists
    ,blocks
    ,steals
    ,fieldGoalsAttempted
    ,fieldGoalsMade
    ,fieldGoalsPercentage
    ,threePointersAttempted
    ,threePointersMade
    ,threePointersPercentage
    ,freeThrowsAttempted
    ,freeThrowsMade
    ,freeThrowsPercentage
    ,reboundsDefensive
    ,reboundsOffensive
    ,reboundsTotal
    ,foulsPersonal
    ,turnovers
    ,plusMinusPoints
*/

    FROM player_statistics_chunk
    """

    # Execute query for this chunk
    result_chunk = duckdb.query(query).df()

    # Write to CSV (first chunk with header, subsequent chunks without)
    if first_chunk:
        result_chunk.to_csv(output_file, index=False, mode='w')
        first_chunk = False
    else:
        result_chunk.to_csv(output_file, index=False, mode='a', header=False)

    # Update progress
    processed_rows += len(result_chunk)
    print(f"Processed chunk {chunk_num+1} - Total rows: {processed_rows}")

    # Clean up to free memory
    duckdb.unregister('player_statistics_chunk')
    del chunk
    del result_chunk

print(f"All chunks processed. Total rows: {processed_rows}")
print(f"Results saved to {output_file}")


# Read in the player-statistics-new csv
player_statistics_df = pd.read_csv(output_file).head(20)
player_statistics_df


Memory cleared from previous DataFrame

Downloaded player-statistics.csv
Processing data in chunks of 100000 rows...
Processed chunk 1 - Total rows: 100000
All chunks processed. Total rows: 100000
Results saved to player-statistics-new.csv


/tmp/ipykernel_14436/1293491696.py:118: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  player_statistics_df = pd.read_csv(output_file).head(20)


,firstName,lastName,full_name,player_id,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,...,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints
0,Buddy,Hield,Buddy Hield,1627741,22500047,2025-11-14T21:30:00Z,Golden State,Warriors,San Antonio,Spurs,...,0.000,0.0,0.0,0.000,3.0,1.0,4.0,1.0,1.0,-1.0
1,Gary,Payton II,Gary Payton II,1627780,22500047,2025-11-14T21:30:00Z,Golden State,Warriors,San Antonio,Spurs,...,0.500,0.0,0.0,0.000,1.0,2.0,3.0,1.0,2.0,3.0
2,De'Aaron,Fox,De'Aaron Fox,1628368,22500047,2025-11-14T21:30:00Z,San Antonio,Spurs,Golden State,Warriors,...,0.500,0.0,0.0,0.000,1.0,3.0,4.0,3.0,4.0,1.0
3,Luke,Kornet,Luke Kornet,1628436,22500047,2025-11-14T21:30:00Z,San Antonio,Spurs,Golden State,Warriors,...,0.000,2.0,2.0,1.000,3.0,2.0,5.0,1.0,1.0,-10.0
4,Jordan,McLaughlin,Jordan McLaughlin,1629162,22500047,2025-11-14T21:30:00Z,San Antonio,Spurs,Golden State,Warriors,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
5,Keldon,Johnson,Keldon Johnson,1629640,22500047,2025-11-14T21:30:00Z,San Antonio,Spurs,Golden State,Warriors,...,0.000,0.0,0.0,0.000,5.0,0.0,5.0,2.0,1.0,2.0
6,Devin,Vassell,Devin Vassell,1630170,22500047,2025-11-14T21:30:00Z,San Antonio,Spurs,Golden State,Warriors,...,0.250,2.0,2.0,1.000,3.0,0.0,3.0,3.0,2.0,-16.0
7,Jonathan,Kuminga,Jonathan Kuminga,1630228,22500047,2025-11-14T21:30:00Z,Golden State,Warriors,San Antonio,Spurs,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
8,Pat,Spencer,Pat Spencer,1630311,22500047,2025-11-14T21:30:00Z,Golden State,Warriors,San Antonio,Spurs,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
9,Lindy,Waters III,Lindy Waters III,1630322,22500047,2025-11-14T21:30:00Z,San Antonio,Spurs,Golden State,Warriors,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:

query = """
SELECT
firstName
,lastName
,full_name
,player_id
,gameId
,CAST(gameDate AS DATE) as gameDate
,playerteamCity
,playerteamName
,opponentteamCity
,opponentteamName
,gameType
,gameLabel
,gameSubLabel
,seriesGameNumber
,win
,home
,numMinutes
,points
,assists
,blocks
,steals
,fieldGoalsAttempted
,fieldGoalsMade
,fieldGoalsPercentage
,threePointersAttempted
,threePointersMade
,threePointersPercentage
,freeThrowsAttempted
,freeThrowsMade
,freeThrowsPercentage
,reboundsDefensive
,reboundsOffensive
,reboundsTotal
,foulsPersonal
,turnovers
,plusMinusPoints
FROM
player_statistics_df
"""

duckdb.query(query).df()

,firstName,lastName,full_name,player_id,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,...,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints
0,Buddy,Hield,Buddy Hield,1627741,22500047,2025-11-14,Golden State,Warriors,San Antonio,Spurs,...,0.000,0.0,0.0,0.000,3.0,1.0,4.0,1.0,1.0,-1.0
1,Gary,Payton II,Gary Payton II,1627780,22500047,2025-11-14,Golden State,Warriors,San Antonio,Spurs,...,0.500,0.0,0.0,0.000,1.0,2.0,3.0,1.0,2.0,3.0
2,De'Aaron,Fox,De'Aaron Fox,1628368,22500047,2025-11-14,San Antonio,Spurs,Golden State,Warriors,...,0.500,0.0,0.0,0.000,1.0,3.0,4.0,3.0,4.0,1.0
3,Luke,Kornet,Luke Kornet,1628436,22500047,2025-11-14,San Antonio,Spurs,Golden State,Warriors,...,0.000,2.0,2.0,1.000,3.0,2.0,5.0,1.0,1.0,-10.0
4,Jordan,McLaughlin,Jordan McLaughlin,1629162,22500047,2025-11-14,San Antonio,Spurs,Golden State,Warriors,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
5,Keldon,Johnson,Keldon Johnson,1629640,22500047,2025-11-14,San Antonio,Spurs,Golden State,Warriors,...,0.000,0.0,0.0,0.000,5.0,0.0,5.0,2.0,1.0,2.0
6,Devin,Vassell,Devin Vassell,1630170,22500047,2025-11-14,San Antonio,Spurs,Golden State,Warriors,...,0.250,2.0,2.0,1.000,3.0,0.0,3.0,3.0,2.0,-16.0
7,Jonathan,Kuminga,Jonathan Kuminga,1630228,22500047,2025-11-14,Golden State,Warriors,San Antonio,Spurs,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
8,Pat,Spencer,Pat Spencer,1630311,22500047,2025-11-14,Golden State,Warriors,San Antonio,Spurs,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
9,Lindy,Waters III,Lindy Waters III,1630322,22500047,2025-11-14,San Antonio,Spurs,Golden State,Warriors,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import wget
import pandas as pd

In [ ]:
# Download files
print("Downloading files...")
filename = 'nba-all-stars.csv'
url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
wget.download(url)
print(f"\nDownloaded {filename}")


Downloaded nba-all-stars.csv


In [ ]:
nba_all_stars_df = pd.read_csv('nba-all-stars.csv')
nba_all_stars_df

,Player,Total Selections,Selections
0,LeBron James,21,2005–2025
1,Kareem Abdul-Jabbar,19,1970–1977; 1979–1989
2,Kobe Bryant,18,1998; 2000–2016
3,Tim Duncan,15,1998; 2000–2011; 2013; 2015
4,Kevin Durant,15,2010–2019; 2021–2025
...,...,...,...
455,Jayson Williams,1,1998
456,Mo Williams,1,2009
457,Kevin Willis,1,1992
458,Metta World Peace,1,2004


In [ ]:
import pandas as pd
import duckdb
import unicodedata

def remove_accents(text):
    """
    Remove accent marks from input text while preserving the base characters.
    Also handles special characters like Đ/đ.

    Example:
    "Nikola Đurišić" -> "Nikola Durisic"
    """
    # First, handle special characters that need specific replacements
    special_chars = {
        'Đ': 'D', 'đ': 'd',  # Serbian/Croatian D with stroke
        'Ł': 'L', 'ł': 'l',  # Polish L with stroke
        'Ø': 'O', 'ø': 'o',  # Danish/Norwegian O with stroke
        'Ŧ': 'T', 'ŧ': 't',  # Sami T with stroke
        'Æ': 'AE', 'æ': 'ae',  # Æ/æ ligature
        'Œ': 'OE', 'œ': 'oe',  # Œ/œ ligature
        'ß': 'ss',  # German eszett
    }

    for char, replacement in special_chars.items():
        text = text.replace(char, replacement)

    # Normalize the text to decompose characters into base character and accent mark
    normalized_text = unicodedata.normalize('NFKD', text)

    # Filter out the non-spacing marks (accent marks)
    result = ''.join(c for c in normalized_text if not unicodedata.category(c).startswith('Mn'))

    return result

def expand_selection_years(selection_years_string):
    '''
    Takes in a Dataframe column of years summarized and processes it to create a new dataframe column with
    all years spelled out
    '''
    # Normalize non-breaking spaces to regular spaces
    selection_years_string = selection_years_string.replace('\xa0', ' ')
    dash_characters = ["-", "–", "—"] # Hyphen-minus, En Dash, Em Dash
    selections_list = selection_years_string.split("; ")

    consecutive_selection_years = []
    consecutive_selection_years_delimiters = []
    non_consecutive_selection_years = []
    non_consecutive_selection_years_delimiters = []

    # Generate a df for consecutive selection years
    for i in selections_list:
        for dash in dash_characters:
            if dash in i:
                consecutive_selection_years.append(i)
                consecutive_selection_years_delimiters.append(dash)
            else:
                non_consecutive_selection_years.append(i)
    non_consecutive_selection_years = list(set(non_consecutive_selection_years) - set(consecutive_selection_years))
    consecutive_selection_years_df = pd.DataFrame(zip(consecutive_selection_years,consecutive_selection_years_delimiters), columns=['selection_years', 'delimiter'])

    # Generate a df for non-consecutive selection years
    for i in non_consecutive_selection_years:
        non_consecutive_selection_years_delimiters.append("not_applicable")
    non_consecutive_selection_years_df = pd.DataFrame(zip(non_consecutive_selection_years,non_consecutive_selection_years_delimiters), columns=['selection_years', 'delimiter'])

    # Vertically concatenate both dataframes
    selection_years_df = pd.concat([consecutive_selection_years_df,non_consecutive_selection_years_df])

    query = """
    SELECT *
    ,CASE WHEN delimiter != 'not_applicable' THEN LEFT(selection_years, POSITION(delimiter IN selection_years) - 1) END AS min_year
    ,CASE WHEN delimiter != 'not_applicable' THEN RIGHT(selection_years, POSITION(delimiter IN selection_years) - 1) END AS max_year
    FROM selection_years_df
    """

    output = duckdb.query(query).df()

    # Extract all years that aren't within a range of numbers
    single_years = output[output.delimiter == "not_applicable"].selection_years.to_list()
    single_years = [int(year) for year in single_years]

    # Extract all years that are within a range of numbers
    selection_range_years = []
    selection_range = output[output.delimiter != "not_applicable"][['min_year','max_year']].values.tolist()
    for row in selection_range:
        for i in range(int(row[0]),int(row[1]) + 1):
            selection_range_years.append(i)

    # Combine lists
    all_years = list(set(single_years + selection_range_years))

    return all_years

def process_nba_all_stars(csv_file):
    nba_all_stars_df = pd.read_csv(csv_file)
    nba_all_stars_df["Selection Years"] = nba_all_stars_df["Selections"].apply(lambda x: expand_selection_years(x))
    nba_all_stars_df = nba_all_stars_df[['Player', 'Total Selections', 'Selection Years']]
    nba_all_stars_df = nba_all_stars_df.explode('Selection Years').reset_index(drop=True).rename(columns={'Selection Years':'Selection Year'})
    nba_all_stars_df ["Player"] = nba_all_stars_df["Player"].apply(remove_accents)
    nba_all_stars_df.to_csv('nba-all-stars.csv', index=False)

    return nba_all_stars_df

filename = 'nba-all-stars-raw.csv'
nba_all_stars_df = process_nba_all_stars(filename)
nba_all_stars_df

,Player,Total Selections,Selection Year
0,LeBron James,21,2005
1,LeBron James,21,2006
2,LeBron James,21,2007
3,LeBron James,21,2008
4,LeBron James,21,2009
...,...,...,...
1816,Jayson Williams,1,1998
1817,Mo Williams,1,2009
1818,Kevin Willis,1,1992
1819,Metta World Peace,1,2004


Create a function that takes in a DF with NBA names and processes to get back a DF with player_ids attached.

1. Remove accents
2. Left Join original df to nba_mappings
3. Left Join the result from Step 2 to nba_player_lookup

In [ ]:
import pandas as pd
import duckdb
import wget
import unicodedata
import os

def remove_accents(text):
    """
    Remove accent marks from input text while preserving the base characters.
    Also handles special characters like Đ/đ.

    Example:
    "Nikola Đurišić" -> "Nikola Durisic"
    """
    # First, handle special characters that need specific replacements
    special_chars = {
        'Đ': 'D', 'đ': 'd',  # Serbian/Croatian D with stroke
        'Ł': 'L', 'ł': 'l',  # Polish L with stroke
        'Ø': 'O', 'ø': 'o',  # Danish/Norwegian O with stroke
        'Ŧ': 'T', 'ŧ': 't',  # Sami T with stroke
        'Æ': 'AE', 'æ': 'ae',  # Æ/æ ligature
        'Œ': 'OE', 'œ': 'oe',  # Œ/œ ligature
        'ß': 'ss',  # German eszett
    }

    for char, replacement in special_chars.items():
        text = text.replace(char, replacement)

    # Normalize the text to decompose characters into base character and accent mark
    normalized_text = unicodedata.normalize('NFKD', text)

    # Filter out the non-spacing marks (accent marks)
    result = ''.join(c for c in normalized_text if not unicodedata.category(c).startswith('Mn'))

    return result


def attach_player_ids(input_df, column):
    '''
    This function takes in a DataFrame containing NBA player names but not player_id column.
    It outputs a new DataFrame with a player_id column.
    '''
    column = str(column)

    ##### Step 1: Remove all accents from the player's names #####

    input_df[column] = input_df[column].apply(remove_accents)
    #input_df.column = input_df.column.apply(remove_accents)

    ##### Step 2: Left join the df to the nba_mappings lookup table #####
    # Bring in name mapping table for names to help match all names to the format seen in the NBA API
    filename = 'name_mappings.csv'
    url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
    wget.download(url)
    # Read in the name_mappings csv
    name_mapping_df = pd.read_csv('name_mappings.csv')

    #Perform left join
    query = f"""
    WITH CTE AS (
    SELECT * FROM input_df
    LEFT JOIN name_mapping_df a
    ON input_df.{column} = a.in_table_name
    LEFT JOIN name_mapping_df b
    ON input_df.{column} = b.nba_lookup_name
    )
    SELECT *
    ,CASE WHEN nba_lookup_name IS NOT NULL THEN nba_lookup_name ELSE {column} END AS player_name_to_use
    FROM CTE
    """
    input_df = duckdb.query(query).df()

    ##### Step 3: Left join the result from Step 2 to nba_player_lookup table #####
    # Bring in nba player lookup table to map the cleaned names to player IDs. Same player IDs from the NBA API.
    filename = 'nba_player_lookup.csv'
    url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
    wget.download(url)
    # Read in the nba_player_lookup csv
    nba_player_lookup_df = pd.read_csv(filename)
    # Clean each player's full name
    nba_player_lookup_df["player_name"] = nba_player_lookup_df["player_name"].apply(remove_accents)

    #Perform left join
    query = f"""
    WITH CTE AS (
    SELECT
    nba_player_lookup_df.player_id
    ,input_df.*
    FROM input_df
    LEFT JOIN nba_player_lookup_df
    ON input_df.{column} = nba_player_lookup_df.player_name
    )
    SELECT *
    ,CASE
        WHEN player_id IS NOT NULL THEN player_id
        WHEN player_id IS NULL THEN player_id_1
        ELSE player_id_3
    END AS player_id_to_use
    FROM CTE

    """
    output_df = duckdb.query(query).df()
    #Drop unneeded columns and rearrange
    output_df = output_df.drop(columns=[f'{column}','player_id','in_table_name','nba_lookup_name','player_id_1','in_table_name_1','nba_lookup_name_2','player_id_3'])
    output_df = output_df.rename(columns={'player_name_to_use':'player_name', 'player_id_to_use':'player_id'})
    cols = output_df.columns.tolist()
    cols.pop(cols.index('player_name'))
    cols.pop(cols.index('player_id'))
    cols.insert(0, 'player_id')
    cols.insert(1, 'player_name')
    output_df = output_df[cols]

    #Delete lookup tables
    os.remove("name_mappings.csv")
    os.remove("nba_player_lookup.csv")

    return output_df

def expand_selection_years(selection_years_string):
    '''
    Takes in a Dataframe column of years summarized and processes it to create a new dataframe column with
    all years spelled out
    '''
    # Normalize non-breaking spaces to regular spaces
    selection_years_string = selection_years_string.replace('\xa0', ' ')
    dash_characters = ["-", "–", "—"] # Hyphen-minus, En Dash, Em Dash
    selections_list = selection_years_string.split("; ")

    consecutive_selection_years = []
    consecutive_selection_years_delimiters = []
    non_consecutive_selection_years = []
    non_consecutive_selection_years_delimiters = []

    # Generate a df for consecutive selection years
    for i in selections_list:
        for dash in dash_characters:
            if dash in i:
                consecutive_selection_years.append(i)
                consecutive_selection_years_delimiters.append(dash)
            else:
                non_consecutive_selection_years.append(i)
    non_consecutive_selection_years = list(set(non_consecutive_selection_years) - set(consecutive_selection_years))
    consecutive_selection_years_df = pd.DataFrame(zip(consecutive_selection_years,consecutive_selection_years_delimiters), columns=['selection_years', 'delimiter'])

    # Generate a df for non-consecutive selection years
    for i in non_consecutive_selection_years:
        non_consecutive_selection_years_delimiters.append("not_applicable")
    non_consecutive_selection_years_df = pd.DataFrame(zip(non_consecutive_selection_years,non_consecutive_selection_years_delimiters), columns=['selection_years', 'delimiter'])

    # Vertically concatenate both dataframes
    selection_years_df = pd.concat([consecutive_selection_years_df,non_consecutive_selection_years_df])

    query = """
    SELECT *
    ,CASE WHEN delimiter != 'not_applicable' THEN LEFT(selection_years, POSITION(delimiter IN selection_years) - 1) END AS min_year
    ,CASE WHEN delimiter != 'not_applicable' THEN RIGHT(selection_years, POSITION(delimiter IN selection_years) - 1) END AS max_year
    FROM selection_years_df
    """

    output = duckdb.query(query).df()

    # Extract all years that aren't within a range of numbers
    single_years = output[output.delimiter == "not_applicable"].selection_years.to_list()
    single_years = [int(year) for year in single_years]

    # Extract all years that are within a range of numbers
    selection_range_years = []
    selection_range = output[output.delimiter != "not_applicable"][['min_year','max_year']].values.tolist()
    for row in selection_range:
        for i in range(int(row[0]),int(row[1]) + 1):
            selection_range_years.append(i)

    # Combine lists
    all_years = list(set(single_years + selection_range_years))

    return all_years

def process_nba_all_stars(csv_file):
    nba_all_stars_df = pd.read_csv(csv_file)
    nba_all_stars_df["Selection Years"] = nba_all_stars_df["Selections"].apply(lambda x: expand_selection_years(x))
    nba_all_stars_df = nba_all_stars_df[['Player', 'Total Selections', 'Selection Years']]
    nba_all_stars_df = nba_all_stars_df.explode('Selection Years').reset_index(drop=True).rename(columns={'Selection Years':'Selection Year'})
    nba_all_stars_df ["Player"] = nba_all_stars_df["Player"].apply(remove_accents)
    nba_all_stars_df.to_csv('nba-all-stars.csv', index=False)

    return nba_all_stars_df

filename = 'nba-all-stars-raw.csv'
nba_all_stars_df = process_nba_all_stars(filename)
output_df = attach_player_ids(nba_all_stars_df,'Player')
output_df.to_csv('nba-all-stars.csv', index=False)
output_df

,player_id,player_name,Total Selections,Selection Year
0,2544,LeBron James,21,2005
1,2544,LeBron James,21,2006
2,2544,LeBron James,21,2007
3,2544,LeBron James,21,2008
4,2544,LeBron James,21,2009
...,...,...,...,...
1816,913,Larry D. Johnson,2,1995
1817,77376,Lafayette Lever,2,1988
1818,77376,Lafayette Lever,2,1990
1819,76753,World Free,1,1980


In [ ]:
# Download files
filename = 'nba-mvp.csv'
print(f"Downloading {filename}...")
url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
wget.download(url)
print(f"\nDownloaded {filename}")


Downloaded nba-mvp.csv


In [ ]:
nba_mvp_df = pd.read_csv('nba-mvp.csv')
nba_mvp_df

,Season,Player,Team,Pos,Height,Weight,Age,Pre-Draft Team,Draft Yr,YOS
0,2024-2025,Shai Gilgeous-Alexander,Oklahoma City Thunder,PG,6-Jun,195,26,Kentucky,2018,6
1,2023-2024,Nikola Jokic,Denver Nuggets,C,11-Jun,284,29,KK Mega Bemax (Serbia),2014,8
2,2022-2023,Joel Embiid,Philadelphia Sixers,C,Jul-00,280,28,Kansas,2014,8
3,2021-2022,Nikola Jokic,Denver Nuggets,C,11-Jun,284,27,KK Mega Bemax (Serbia),2014,6
4,2020-2021,Nikola Jokic,Denver Nuggets,C,11-Jun,284,26,KK Mega Bemax (Serbia),2014,5
...,...,...,...,...,...,...,...,...,...,...
65,1959-1960,Wilt Chamberlain,San Francisco Warriors,C,1-Jul,265,23,Kansas,1959,0
66,1958-1959,Bob Pettit,St. Louis Hawks,FC,9-Jun,205,26,LSU,1954,4
67,1957-1958,Bill Russell,Boston Celtics,C,10-Jun,215,24,San Francisco,1956,1
68,1956-1957,Bob Cousy,Boston Celtics,PG,1-Jun,175,28,Holy Cross,1950,6


In [ ]:
# Download files
filename = 'nba_player_lookup.csv'
print(f"Downloading {filename}...")
url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
wget.download(url)
print(f"\nDownloaded {filename}")


Downloaded nba_player_lookup.csv


In [ ]:
# Download files
filename = 'all-nba-first-team.csv'
print(f"Downloading {filename}...")
url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
wget.download(url)
print(f"\nDownloaded {filename}")

# Download files
filename = 'all-nba-second-team.csv'
print(f"Downloading {filename}...")
url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
wget.download(url)
print(f"\nDownloaded {filename}")

# Download files
filename = 'all-nba-third-team.csv'
print(f"Downloading {filename}...")
url = f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}'
wget.download(url)
print(f"\nDownloaded {filename}")


Downloaded all-nba-first-team.csv

Downloaded all-nba-second-team.csv

Downloaded all-nba-third-team.csv


In [ ]:
all_nba_first_team_df = pd.read_csv('all-nba-first-team.csv')
all_nba_first_team_df

,Season,Player,Team,Pos,Height,Weight,Age,Pre-Draft Team,Draft Yr,YOS
0,2024-2025,Giannis Antetokounmpo,Milwaukee Bucks,F,11-Jun,243,30,Filathlitikos Div II Greece (Greece),2013,11
1,2024-2025,Shai Gilgeous-Alexander,Oklahoma City Thunder,PG,6-Jun,195,26,Kentucky,2018,6
2,2024-2025,Nikola Jokic,Denver Nuggets,C,11-Jun,284,30,KK Mega Bemax (Serbia),2014,9
3,2024-2025,Donovan Mitchell,Cleveland Cavaliers,SG,2-Jun,215,28,Louisville,2017,7
4,2024-2025,Jayson Tatum,Boston Celtics,SF,8-Jun,210,26,Duke,2017,7
...,...,...,...,...,...,...,...,...,...,...
391,1946-1947,Bob Feerick,Washington Capitols,GF,3-Jun,190,27,NaN,1946,0
392,1946-1947,Joe Fulks,San Francisco Warriors,SF,5-Jun,190,25,NaN,1946,0
393,1946-1947,Bones McKinney,Washington Capitols,FC,6-Jun,185,28,NaN,1946,0
394,1946-1947,Stan Miasek,Detroit Falcons,F,5-Jun,210,22,NaN,1946,0


In [ ]:
all_nba_first_team_df.columns

Index(['Season', 'Player', 'Team', 'Pos', 'Height', 'Weight', 'Age',
       'Pre-Draft Team', 'Draft Yr', 'YOS'],
      dtype='object')

In [ ]:
query = """
SELECT
Season
,Player
,Team
,Pos
,CASE
    WHEN Height = '1-May' THEN '5-1'
    WHEN Height = '2-May' THEN '5-2'
    WHEN Height = '3-May' THEN '5-3'
    WHEN Height = '4-May' THEN '5-4'
    WHEN Height = '5-May' THEN '5-5'
    WHEN Height = '6-May' THEN '5-6'
    WHEN Height = '7-May' THEN '5-7'
    WHEN Height = '8-May' THEN '5-8'
    WHEN Height = '9-May' THEN '5-9'
    WHEN Height = '10-May' THEN '5-10'
    WHEN Height = '11-May' THEN '5-11'
    WHEN Height = 'Jun-00' THEN '6-0'
    WHEN Height = '1-Jun' THEN '6-1'
    WHEN Height = '2-Jun' THEN '6-2'
    WHEN Height = '3-Jun' THEN '6-3'
    WHEN Height = '4-Jun' THEN '6-4'
    WHEN Height = '5-Jun' THEN '6-5'
    WHEN Height = '6-Jun' THEN '6-6'
    WHEN Height = '7-Jun' THEN '6-7'
    WHEN Height = '8-Jun' THEN '6-8'
    WHEN Height = '9-Jun' THEN '6-9'
    WHEN Height = '10-Jun' THEN '6-10'
    WHEN Height = '11-Jun' THEN '6-11'
    WHEN Height = 'Jul-00' THEN '7-0'
    WHEN Height = '1-Jul' THEN '7-1'
    WHEN Height = '2-Jul' THEN '7-2'
    WHEN Height = '3-Jul' THEN '7-3'
    WHEN Height = '4-Jul' THEN '7-4'
    WHEN Height = '5-Jul' THEN '7-5'
    WHEN Height = '6-Jul' THEN '7-6'
    WHEN Height = '7-Jul' THEN '7-7'
    WHEN Height = '8-Jul' THEN '7-8'
    WHEN Height = '9-Jul' THEN '7-9'
END AS Height
,Weight
,Age
,"Pre-Draft Team"
,"Draft Yr"
,YOS

FROM all_nba_first_team_df

"""

all_nba_first_team_df = duckdb.query(query).df()
all_nba_first_team_df

,Season,Player,Team,Pos,Height,Weight,Age,Pre-Draft Team,Draft Yr,YOS
0,2024-2025,Giannis Antetokounmpo,Milwaukee Bucks,F,6-11,243,30,Filathlitikos Div II Greece (Greece),2013,11
1,2024-2025,Shai Gilgeous-Alexander,Oklahoma City Thunder,PG,6-6,195,26,Kentucky,2018,6
2,2024-2025,Nikola Jokic,Denver Nuggets,C,6-11,284,30,KK Mega Bemax (Serbia),2014,9
3,2024-2025,Donovan Mitchell,Cleveland Cavaliers,SG,6-2,215,28,Louisville,2017,7
4,2024-2025,Jayson Tatum,Boston Celtics,SF,6-8,210,26,Duke,2017,7
...,...,...,...,...,...,...,...,...,...,...
391,1946-1947,Bob Feerick,Washington Capitols,GF,6-3,190,27,None,1946,0
392,1946-1947,Joe Fulks,San Francisco Warriors,SF,6-5,190,25,None,1946,0
393,1946-1947,Bones McKinney,Washington Capitols,FC,6-6,185,28,None,1946,0
394,1946-1947,Stan Miasek,Detroit Falcons,F,6-5,210,22,None,1946,0


In [ ]:
all_nba_second_team_df = pd.read_csv('all-nba-second-team.csv')
all_nba_third_team_df = pd.read_csv('all-nba-third-team.csv')

In [ ]:
query = """
SELECT
Season
,Player
,Team
,Pos
,CASE
    WHEN Height = '1-May' THEN '5-1'
    WHEN Height = '2-May' THEN '5-2'
    WHEN Height = '3-May' THEN '5-3'
    WHEN Height = '4-May' THEN '5-4'
    WHEN Height = '5-May' THEN '5-5'
    WHEN Height = '6-May' THEN '5-6'
    WHEN Height = '7-May' THEN '5-7'
    WHEN Height = '8-May' THEN '5-8'
    WHEN Height = '9-May' THEN '5-9'
    WHEN Height = '10-May' THEN '5-10'
    WHEN Height = '11-May' THEN '5-11'
    WHEN Height = 'Jun-00' THEN '6-0'
    WHEN Height = '1-Jun' THEN '6-1'
    WHEN Height = '2-Jun' THEN '6-2'
    WHEN Height = '3-Jun' THEN '6-3'
    WHEN Height = '4-Jun' THEN '6-4'
    WHEN Height = '5-Jun' THEN '6-5'
    WHEN Height = '6-Jun' THEN '6-6'
    WHEN Height = '7-Jun' THEN '6-7'
    WHEN Height = '8-Jun' THEN '6-8'
    WHEN Height = '9-Jun' THEN '6-9'
    WHEN Height = '10-Jun' THEN '6-10'
    WHEN Height = '11-Jun' THEN '6-11'
    WHEN Height = 'Jul-00' THEN '7-0'
    WHEN Height = '1-Jul' THEN '7-1'
    WHEN Height = '2-Jul' THEN '7-2'
    WHEN Height = '3-Jul' THEN '7-3'
    WHEN Height = '4-Jul' THEN '7-4'
    WHEN Height = '5-Jul' THEN '7-5'
    WHEN Height = '6-Jul' THEN '7-6'
    WHEN Height = '7-Jul' THEN '7-7'
    WHEN Height = '8-Jul' THEN '7-8'
    WHEN Height = '9-Jul' THEN '7-9'
END AS Height
,Weight
,Age
,"Pre-Draft Team"
,"Draft Yr"
,YOS

FROM all_nba_second_team_df

"""

all_nba_second_team_df = duckdb.query(query).df()
all_nba_second_team_df

,Season,Player,Team,Pos,Height,Weight,Age,Pre-Draft Team,Draft Yr,YOS
0,2024-2025,Jalen Brunson,New York Knicks,PG,6-2,190,28,Villanova,2018,6
1,2024-2025,Stephen Curry,Golden State Warriors,G,6-2,185,36,Davidson,2009,15
2,2024-2025,Anthony Edwards,Minnesota Timberwolves,SF,6-4,225,23,Georgia,2020,4
3,2024-2025,LeBron James,Los Angeles Lakers,F,6-9,250,40,St. Vincent St. Mary High School (Ohio),2003,21
4,2024-2025,Evan Mobley,Cleveland Cavaliers,PF,6-11,215,23,USC,2021,3
...,...,...,...,...,...,...,...,...,...,...
389,1946-1947,Frankie Baumholtz,Cleveland Rebels,PG,5-10,170,28,None,1946,0
390,1946-1947,Ernie Calverley,Providence Steamrollers,PG,5-10,145,23,None,1946,0
391,1946-1947,Chick Halbert,Chicago Stags,PF,6-9,225,28,None,1946,0
392,1946-1947,Johnny Logan,St. Louis Bombers,G,6-2,175,26,None,1946,0


In [ ]:
query = """
SELECT
Season
,Player
,Team
,Pos
,CASE
    WHEN Height = '1-May' THEN '5-1'
    WHEN Height = '2-May' THEN '5-2'
    WHEN Height = '3-May' THEN '5-3'
    WHEN Height = '4-May' THEN '5-4'
    WHEN Height = '5-May' THEN '5-5'
    WHEN Height = '6-May' THEN '5-6'
    WHEN Height = '7-May' THEN '5-7'
    WHEN Height = '8-May' THEN '5-8'
    WHEN Height = '9-May' THEN '5-9'
    WHEN Height = '10-May' THEN '5-10'
    WHEN Height = '11-May' THEN '5-11'
    WHEN Height = 'Jun-00' THEN '6-0'
    WHEN Height = '1-Jun' THEN '6-1'
    WHEN Height = '2-Jun' THEN '6-2'
    WHEN Height = '3-Jun' THEN '6-3'
    WHEN Height = '4-Jun' THEN '6-4'
    WHEN Height = '5-Jun' THEN '6-5'
    WHEN Height = '6-Jun' THEN '6-6'
    WHEN Height = '7-Jun' THEN '6-7'
    WHEN Height = '8-Jun' THEN '6-8'
    WHEN Height = '9-Jun' THEN '6-9'
    WHEN Height = '10-Jun' THEN '6-10'
    WHEN Height = '11-Jun' THEN '6-11'
    WHEN Height = 'Jul-00' THEN '7-0'
    WHEN Height = '1-Jul' THEN '7-1'
    WHEN Height = '2-Jul' THEN '7-2'
    WHEN Height = '3-Jul' THEN '7-3'
    WHEN Height = '4-Jul' THEN '7-4'
    WHEN Height = '5-Jul' THEN '7-5'
    WHEN Height = '6-Jul' THEN '7-6'
    WHEN Height = '7-Jul' THEN '7-7'
    WHEN Height = '8-Jul' THEN '7-8'
    WHEN Height = '9-Jul' THEN '7-9'
END AS Height
,Weight
,Age
,"Pre-Draft Team"
,"Draft Yr"
,YOS

FROM all_nba_third_team_df

"""

all_nba_third_team_df = duckdb.query(query).df()
all_nba_third_team_df

,Season,Player,Team,Pos,Height,Weight,Age,Pre-Draft Team,Draft Yr,YOS
0,2024-2025,Cade Cunningham,Detroit Pistons,SF,6-6,220,23,Oklahoma State,2021,3
1,2024-2025,Tyrese Haliburton,Indiana Pacers,PG,6-5,185,25,Iowa State,2020,4
2,2024-2025,James Harden,Los Angeles Clippers,SG,6-5,220,35,Arizona State,2009,15
3,2024-2025,Karl-Anthony Towns,New York Knicks,C,7-0,248,29,Kentucky,2015,9
4,2024-2025,Jalen Williams,Oklahoma City Thunder,G,6-5,211,23,Santa Clara,2022,2
...,...,...,...,...,...,...,...,...,...,...
180,1988-1989,Terry Cummings,Milwaukee Bucks,PF,6-9,220,27,DePaul,1982,6
181,1988-1989,Dale Ellis,Seattle SuperSonics,GF,6-7,205,28,Tennessee,1983,5
182,1988-1989,Robert Parish,Boston Celtics,C,7-0,230,35,Centenary (LA),1976,11
183,1988-1989,Mark Price,Cleveland Cavaliers,PG,6-0,170,25,Georgia Tech,1986,2


In [ ]:
all_nba_third_team_df.Height.value_counts().sum()

np.int64(185)

In [ ]:
# all_nba_first_team_df.to_csv('all-nba-first-team.csv', index=False)
# all_nba_second_team_df.to_csv('all-nba-second-team.csv', index=False)
# all_nba_third_team_df.to_csv('all-nba-third-team.csv', index=False)

In [ ]:
all_nba_second_team_df = pd.read_csv('all-nba-second-team.csv')

In [ ]:
testing_df_1 = attach_player_ids(all_nba_first_team_df, 'Player')
testing_df_2 = attach_player_ids(all_nba_second_team_df, 'Player')
testing_df_3 = attach_player_ids(all_nba_third_team_df, 'Player')

In [ ]:
query = """
SELECT * FROM testing_df_1
WHERE player_id IS NULL

UNION ALL
SELECT * FROM testing_df_2
WHERE player_id IS NULL

UNION ALL
SELECT * FROM testing_df_3
WHERE player_id IS NULL

"""

duckdb.query(query).df()

,player_id,player_name,Season,Team,Pos,Height,Weight,Age,Pre-Draft Team,Draft Yr,YOS


In [ ]:
query = """
SELECT * FROM testing_df_1
ORDER BY Season DESC
"""

all_nba_first_team_df = duckdb.query(query).df()
all_nba_first_team_df.to_csv('all-nba-first-team.csv',index=False)

In [ ]:
query = """
SELECT * FROM testing_df_2
ORDER BY Season DESC
"""

all_nba_second_team_df = duckdb.query(query).df()
all_nba_second_team_df.to_csv('all-nba-second-team.csv',index=False)

In [ ]:
query = """
SELECT * FROM testing_df_3
ORDER BY Season DESC
"""

all_nba_third_team_df = duckdb.query(query).df()
all_nba_third_team_df.to_csv('all-nba-third-team.csv',index=False)

In [ ]:
player_stats_test_df = pd.read_csv('player-statistics-new.csv')

/tmp/ipykernel_4698/495494136.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  player_stats_test_df = pd.read_csv('player-statistics-new.csv')


In [ ]:
query = """


SELECT *
FROM player_stats_test_df
--LEFT JOIN all_nba_first_team_df
--ON player_stats_test_df.player_id = all_nba_first_team_df.player_id
--LEFT JOIN all_nba_second_team_df
--ON player_stats_test_df.player_id = all_nba_second_team_df.player_id
--LEFT JOIN all_nba_third_team_df
--ON player_stats_test_df.player_id = all_nba_third_team_df.player_id
"""

duckdb.query(query).df()

,firstName,lastName,full_name,player_id,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,...,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints
0,Buddy,Hield,Buddy Hield,1627741,22500047,2025-11-14T21:30:00Z,Golden State,Warriors,San Antonio,Spurs,...,0.000,0.0,0.0,0.0,3.0,1.0,4.0,1.0,1.0,-1.0
1,Gary,Payton II,Gary Payton II,1627780,22500047,2025-11-14T21:30:00Z,Golden State,Warriors,San Antonio,Spurs,...,0.500,0.0,0.0,0.0,1.0,2.0,3.0,1.0,2.0,3.0
2,De'Aaron,Fox,De'Aaron Fox,1628368,22500047,2025-11-14T21:30:00Z,San Antonio,Spurs,Golden State,Warriors,...,0.500,0.0,0.0,0.0,1.0,3.0,4.0,3.0,4.0,1.0
3,Luke,Kornet,Luke Kornet,1628436,22500047,2025-11-14T21:30:00Z,San Antonio,Spurs,Golden State,Warriors,...,0.000,2.0,2.0,1.0,3.0,2.0,5.0,1.0,1.0,-10.0
4,Jordan,McLaughlin,Jordan McLaughlin,1629162,22500047,2025-11-14T21:30:00Z,San Antonio,Spurs,Golden State,Warriors,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Udoka,Azubuike,Udoka Azubuike,1628962,22200646,2023-01-14 21:00:00,Utah,Jazz,Philadelphia,76ers,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99996,De'Anthony,Melton,De'Anthony Melton,1629001,22200646,2023-01-14 21:00:00,Philadelphia,76ers,Utah,Jazz,...,0.667,0.0,0.0,0.0,4.0,0.0,4.0,2.0,1.0,1.0
99997,Shake,Milton,Shake Milton,1629003,22200646,2023-01-14 21:00:00,Philadelphia,76ers,Utah,Jazz,...,0.600,0.0,0.0,0.0,4.0,0.0,4.0,3.0,1.0,3.0
99998,Jarred,Vanderbilt,Jarred Vanderbilt,1629020,22200646,2023-01-14 21:00:00,Utah,Jazz,Philadelphia,76ers,...,0.000,0.0,0.0,0.0,6.0,3.0,9.0,4.0,0.0,-5.0


In [ ]:
all_nba_third_team_df

,player_id,player_name,Season,Team,Pos,Height,Weight,Age,Pre-Draft Team,Draft Yr,YOS
0,1630595,Cade Cunningham,2024-2025,Detroit Pistons,SF,6-6,220,23,Oklahoma State,2021,3
1,1630169,Tyrese Haliburton,2024-2025,Indiana Pacers,PG,6-5,185,25,Iowa State,2020,4
2,1631114,Jalen Williams,2024-2025,Oklahoma City Thunder,G,6-5,211,23,Santa Clara,2022,2
3,201935,James Harden,2024-2025,Los Angeles Clippers,SG,6-5,220,35,Arizona State,2009,15
4,1626157,Karl-Anthony Towns,2024-2025,New York Knicks,C,7-0,248,29,Kentucky,2015,9
...,...,...,...,...,...,...,...,...,...,...,...
180,305,Robert Parish,1988-1989,Boston Celtics,C,7-0,230,35,Centenary (LA),1976,11
181,1122,Dominique Wilkins,1988-1989,Atlanta Hawks,SF,6-7,200,29,Georgia,1982,6
182,107,Dale Ellis,1988-1989,Seattle SuperSonics,GF,6-7,205,28,Tennessee,1983,5
183,899,Mark Price,1988-1989,Cleveland Cavaliers,PG,6-0,170,25,Georgia Tech,1986,2


In [16]:
import pandas as pd
import numpy as np
import duckdb
import wget
import os

wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/nba-all-stars.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/nba-mvp.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/all-nba-first-team.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/all-nba-second-team.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/all-nba-third-team.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/player-of-the-week.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/player-statistics.csv')

player_statistics_test_df = pd.read_csv('player-statistics.csv')

/tmp/ipython-input-2394170347.py:15: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  player_statistics_test_df = pd.read_csv('player-statistics.csv')


In [17]:
query = """

SELECT *,
    CASE
        WHEN gameDate BETWEEN '2025-10-02' AND '2026-08-01' THEN 2025
        WHEN gameDate BETWEEN '2024-10-04' AND '2025-06-22' THEN 2024
        WHEN gameDate BETWEEN '2023-10-05' AND '2024-06-17' THEN 2023
        WHEN gameDate BETWEEN '2022-09-30' AND '2023-06-12' THEN 2022
        WHEN gameDate BETWEEN '2021-10-03' AND '2022-06-16' THEN 2021
        WHEN gameDate BETWEEN '2020-12-11' AND '2021-07-20' THEN 2020
        WHEN gameDate BETWEEN '2019-10-22' AND '2020-10-11' THEN 2019
        WHEN gameDate BETWEEN '2018-10-16' AND '2019-06-10' THEN 2018
        WHEN gameDate BETWEEN '2017-09-30' AND '2018-06-08' THEN 2017
        WHEN gameDate BETWEEN '2016-10-01' AND '2017-06-12' THEN 2016
        WHEN gameDate BETWEEN '2015-10-02' AND '2016-06-19' THEN 2015
        WHEN gameDate BETWEEN '2014-10-04' AND '2015-06-16' THEN 2014
        WHEN gameDate BETWEEN '2013-10-05' AND '2014-06-15' THEN 2013
        WHEN gameDate BETWEEN '2012-10-05' AND '2013-06-20' THEN 2012
        WHEN gameDate BETWEEN '2011-12-16' AND '2012-06-21' THEN 2011
        WHEN gameDate BETWEEN '2010-10-03' AND '2011-06-12' THEN 2010
        WHEN gameDate BETWEEN '2009-10-01' AND '2010-06-17' THEN 2009
        WHEN gameDate BETWEEN '2008-10-05' AND '2009-06-11' THEN 2008
        WHEN gameDate BETWEEN '2007-10-06' AND '2008-06-17' THEN 2007
        WHEN gameDate BETWEEN '2006-10-05' AND '2007-06-14' THEN 2006
        WHEN gameDate BETWEEN '2005-10-10' AND '2006-04-19' THEN 2005
        WHEN gameDate BETWEEN '2004-11-02' AND '2005-06-21' THEN 2004
        WHEN gameDate BETWEEN '2003-10-28' AND '2004-06-15' THEN 2003
        WHEN gameDate BETWEEN '2002-10-29' AND '2003-06-15' THEN 2002
        WHEN gameDate BETWEEN '2001-10-30' AND '2002-04-17' THEN 2001
        WHEN gameDate BETWEEN '2000-10-31' AND '2001-06-15' THEN 2000
        WHEN gameDate BETWEEN '1999-11-02' AND '2000-04-19' THEN 1999
        WHEN gameDate BETWEEN '1998-02-05' AND '1999-06-25' THEN 1998
        WHEN gameDate BETWEEN '1997-10-31' AND '1998-06-14' THEN 1997
        WHEN gameDate BETWEEN '1996-11-01' AND '1997-06-13' THEN 1996
        WHEN gameDate BETWEEN '1995-11-03' AND '1996-04-21' THEN 1995
        WHEN gameDate BETWEEN '1994-11-04' AND '1995-06-14' THEN 1994
        WHEN gameDate BETWEEN '1993-11-05' AND '1994-04-24' THEN 1993
        WHEN gameDate BETWEEN '1992-11-06' AND '1993-06-20' THEN 1992
        WHEN gameDate BETWEEN '1991-11-01' AND '1992-06-12' THEN 1991
        WHEN gameDate BETWEEN '1990-11-02' AND '1991-06-12' THEN 1990
        WHEN gameDate BETWEEN '1989-11-03' AND '1990-06-14' THEN 1989
        WHEN gameDate BETWEEN '1988-11-04' AND '1989-06-13' THEN 1988
        WHEN gameDate BETWEEN '1987-11-06' AND '1988-06-19' THEN 1987
        WHEN gameDate BETWEEN '1986-10-31' AND '1987-06-14' THEN 1986
        WHEN gameDate BETWEEN '1985-10-25' AND '1986-06-05' THEN 1985
        WHEN gameDate BETWEEN '1984-10-26' AND '1985-06-09' THEN 1984
        WHEN gameDate BETWEEN '1983-10-28' AND '1984-06-12' THEN 1983
        WHEN gameDate BETWEEN '1982-10-29' AND '1983-05-31' THEN 1982
        WHEN gameDate BETWEEN '1981-10-30' AND '1982-06-06' THEN 1981
        WHEN gameDate BETWEEN '1980-10-10' AND '1981-05-14' THEN 1980
        WHEN gameDate BETWEEN '1979-10-12' AND '1980-05-16' THEN 1979
        WHEN gameDate BETWEEN '1978-10-13' AND '1979-06-01' THEN 1978
        WHEN gameDate BETWEEN '1977-10-18' AND '1978-06-07' THEN 1977
        WHEN gameDate BETWEEN '1976-02-13' AND '1977-06-05' THEN 1976
        WHEN gameDate BETWEEN '1976-02-03' AND '1976-06-06' THEN 1975
        WHEN gameDate BETWEEN '1974-10-17' AND '1975-05-25' THEN 1974
        WHEN gameDate BETWEEN '1973-10-09' AND '1974-05-12' THEN 1973
        WHEN gameDate BETWEEN '1972-10-10' AND '1973-05-10' THEN 1972
        WHEN gameDate BETWEEN '1971-10-12' AND '1972-05-07' THEN 1971
        WHEN gameDate BETWEEN '1971-01-12' AND '1971-04-30' THEN 1970
        WHEN gameDate BETWEEN '1969-10-14' AND '1970-05-08' THEN 1969
        WHEN gameDate BETWEEN '1968-10-15' AND '1969-03-24' THEN 1968
        WHEN gameDate BETWEEN '1967-10-13' AND '1968-05-02' THEN 1967
        WHEN gameDate BETWEEN '1967-01-10' AND '1967-04-24' THEN 1966
        WHEN gameDate BETWEEN '1965-10-15' AND '1966-04-28' THEN 1965
        WHEN gameDate BETWEEN '1964-10-16' AND '1965-03-21' THEN 1964
        WHEN gameDate BETWEEN '1963-10-16' AND '1964-04-26' THEN 1963
        WHEN gameDate BETWEEN '1962-10-19' AND '1963-04-24' THEN 1962
        WHEN gameDate BETWEEN '1961-02-16' AND '1962-04-18' THEN 1961
        WHEN gameDate BETWEEN '1961-01-17' AND '1961-01-17' THEN 1960
        WHEN gameDate BETWEEN '1959-10-18' AND '1960-04-09' THEN 1959
        WHEN gameDate BETWEEN '1958-10-19' AND '1959-03-11' THEN 1958
        WHEN gameDate BETWEEN '1957-10-22' AND '1958-03-12' THEN 1957
        WHEN gameDate BETWEEN '1956-10-27' AND '1957-04-13' THEN 1956
        WHEN gameDate BETWEEN '1955-11-05' AND '1956-04-05' THEN 1955
        WHEN gameDate BETWEEN '1954-10-30' AND '1955-04-10' THEN 1954
        WHEN gameDate BETWEEN '1953-10-30' AND '1954-04-11' THEN 1953
        WHEN gameDate BETWEEN '1952-10-31' AND '1953-04-10' THEN 1952
        WHEN gameDate BETWEEN '1951-11-01' AND '1952-04-23' THEN 1951
        WHEN gameDate BETWEEN '1950-10-31' AND '1951-04-21' THEN 1950
        WHEN gameDate BETWEEN '1949-10-29' AND '1950-04-23' THEN 1949
        WHEN gameDate BETWEEN '1948-11-01' AND '1949-04-13' THEN 1948
        WHEN gameDate BETWEEN '1947-11-12' AND '1948-04-21' THEN 1947
        WHEN gameDate BETWEEN '1946-11-01' AND '1947-04-22' THEN 1946
    END AS season
FROM player_statistics_test_df


"""

player_statistics_test_df = duckdb.query(query).df()
player_statistics_test_df

,firstName,lastName,full_name,player_id,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,...,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints,season
0,Myles,Turner,Myles Turner,1626167,22500231,2025-11-15T20:00:00Z,Milwaukee,Bucks,Los Angeles,Lakers,...,0.0,0.0,0.000,7.0,1.0,8.0,3.0,1.0,-12.0,2025
1,Bobby,Portis,Bobby Portis,1626171,22500231,2025-11-15T20:00:00Z,Milwaukee,Bucks,Los Angeles,Lakers,...,0.0,0.0,0.000,4.0,2.0,6.0,2.0,0.0,-2.0,2025
2,Jamal,Murray,Jamal Murray,1627750,22500232,2025-11-15T20:00:00Z,Denver,Nuggets,Minnesota,Timberwolves,...,6.0,4.0,0.667,6.0,0.0,6.0,0.0,3.0,6.0,2025
3,Kyle,Kuzma,Kyle Kuzma,1628398,22500231,2025-11-15T20:00:00Z,Milwaukee,Bucks,Los Angeles,Lakers,...,2.0,1.0,0.500,1.0,0.0,1.0,5.0,1.0,-17.0,2025
4,Maxi,Kleber,Maxi Kleber,1628467,22500231,2025-11-15T20:00:00Z,Los Angeles,Lakers,Milwaukee,Bucks,...,0.0,0.0,0.000,1.0,0.0,1.0,5.0,0.0,22.0,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635040,Fred,Sheffield,Fred Sheffield,78131,24600052,1946-11-26 19:00:00,Philadelphia,Warriors,Boston,Celtics,...,0.0,1.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1946
1635041,Connie,Simmons,Connie Simmons,78153,24600052,1946-11-26 19:00:00,Boston,Celtics,Philadelphia,Warriors,...,0.0,5.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1946
1635042,Johnny,Simmons,Johnny Simmons,78154,24600052,1946-11-26 19:00:00,Boston,Celtics,Philadelphia,Warriors,...,0.0,2.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1946
1635043,Virgil,Vaughn,Virgil Vaughn,78411,24600052,1946-11-26 19:00:00,Boston,Celtics,Philadelphia,Warriors,...,0.0,2.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1946


In [18]:
nba_all_stars_df = pd.read_csv('nba-all-stars.csv')
nba_all_stars_df["Season"] = nba_all_stars_df["Selection Year"] - 1
nba_all_stars_df["player_id"] = nba_all_stars_df["player_id"].astype(int)
nba_all_stars_df

,player_id,player_name,Total Selections,Selection Year,Season
0,2544,LeBron James,21,2005,2004
1,2544,LeBron James,21,2006,2005
2,2544,LeBron James,21,2007,2006
3,2544,LeBron James,21,2008,2007
4,2544,LeBron James,21,2009,2008
...,...,...,...,...,...
1816,913,Larry D. Johnson,2,1995,1994
1817,77376,Lafayette Lever,2,1988,1987
1818,77376,Lafayette Lever,2,1990,1989
1819,76753,World Free,1,1980,1979


In [19]:
nba_mvp_df = pd.read_csv('nba-mvp.csv')
nba_mvp_df["Season"] = nba_mvp_df["Season"].str[:4].astype(int)
nba_mvp_df

,player_id,player_name,Season,Team,Pos,Height,Weight,Age,Pre-Draft Team,Draft Yr,YOS
0,1628983,Shai Gilgeous-Alexander,2024,Oklahoma City Thunder,PG,6-6,195,26,Kentucky,2018,6
1,203999,Nikola Jokic,2023,Denver Nuggets,C,6-11,284,29,KK Mega Bemax (Serbia),2014,8
2,203954,Joel Embiid,2022,Philadelphia Sixers,C,7-0,280,28,Kansas,2014,8
3,203999,Nikola Jokic,2021,Denver Nuggets,C,6-11,284,27,KK Mega Bemax (Serbia),2014,6
4,203999,Nikola Jokic,2020,Denver Nuggets,C,6-11,284,26,KK Mega Bemax (Serbia),2014,5
...,...,...,...,...,...,...,...,...,...,...,...
65,76375,Wilt Chamberlain,1959,San Francisco Warriors,C,7-1,265,23,Kansas,1959,0
66,77847,Bob Pettit,1958,St. Louis Hawks,FC,6-9,205,26,LSU,1954,4
67,78049,Bill Russell,1957,Boston Celtics,C,6-10,215,24,San Francisco,1956,1
68,600003,Bob Cousy,1956,Boston Celtics,PG,6-1,175,28,Holy Cross,1950,6


In [20]:
all_nba_first_team_df = pd.read_csv('all-nba-first-team.csv')
all_nba_first_team_df["Season"] = all_nba_first_team_df["Season"].str[:4].astype(int)
all_nba_first_team_df

all_nba_second_team_df = pd.read_csv('all-nba-second-team.csv')
all_nba_second_team_df["Season"] = all_nba_second_team_df["Season"].str[:4].astype(int)
all_nba_second_team_df

all_nba_third_team_df = pd.read_csv('all-nba-third-team.csv')
all_nba_third_team_df["Season"] = all_nba_third_team_df["Season"].str[:4].astype(int)
all_nba_third_team_df

,player_id,player_name,Season,Team,Pos,Height,Weight,Age,Pre-Draft Team,Draft Yr,YOS
0,1630595,Cade Cunningham,2024,Detroit Pistons,SF,6-6,220,23,Oklahoma State,2021,3
1,1630169,Tyrese Haliburton,2024,Indiana Pacers,PG,6-5,185,25,Iowa State,2020,4
2,1631114,Jalen Williams,2024,Oklahoma City Thunder,G,6-5,211,23,Santa Clara,2022,2
3,201935,James Harden,2024,Los Angeles Clippers,SG,6-5,220,35,Arizona State,2009,15
4,1626157,Karl-Anthony Towns,2024,New York Knicks,C,7-0,248,29,Kentucky,2015,9
...,...,...,...,...,...,...,...,...,...,...,...
180,305,Robert Parish,1988,Boston Celtics,C,7-0,230,35,Centenary (LA),1976,11
181,1122,Dominique Wilkins,1988,Atlanta Hawks,SF,6-7,200,29,Georgia,1982,6
182,107,Dale Ellis,1988,Seattle SuperSonics,GF,6-7,205,28,Tennessee,1983,5
183,899,Mark Price,1988,Cleveland Cavaliers,PG,6-0,170,25,Georgia Tech,1986,2


In [21]:
query = """

WITH All_Star_Flag AS
(
SELECT *,
'Yes' AS all_star_this_season
FROM nba_all_stars_df
)

,MVP_Flag AS (
SELECT *,
'Yes' AS mvp_this_season
FROM nba_mvp_df
)

,All_NBA_First_Team_Flag AS (
SELECT *,
'Yes' AS all_nba_first_team_this_season
FROM all_nba_first_team_df
)

,All_NBA_Second_Team_Flag AS (
SELECT *,
'Yes' AS all_nba_second_team_this_season
FROM all_nba_second_team_df
)

,All_NBA_Third_Team_Flag AS (
SELECT *,
'Yes' AS all_nba_third_team_this_season
FROM all_nba_third_team_df
)

SELECT
player_statistics_test_df.firstName
,player_statistics_test_df.lastName
,player_statistics_test_df.full_name
,player_statistics_test_df.player_id
,player_statistics_test_df.gameId
,CAST(player_statistics_test_df.gameDate AS DATE) AS gameDate
,player_statistics_test_df.playerteamCity
,player_statistics_test_df.playerteamName
,player_statistics_test_df.opponentteamCity
,player_statistics_test_df.opponentteamName
,player_statistics_test_df.gameType
,player_statistics_test_df.gameLabel
,player_statistics_test_df.gameSubLabel
,player_statistics_test_df.seriesGameNumber
,player_statistics_test_df.win
,player_statistics_test_df.home
,player_statistics_test_df.numMinutes
,CAST(player_statistics_test_df.points AS INT) AS points
,CAST(player_statistics_test_df.assists AS INT) AS assists
,CAST(player_statistics_test_df.blocks AS INT) AS blocks
,CAST(player_statistics_test_df.steals AS INT) AS steals
,CAST(player_statistics_test_df.fieldGoalsAttempted AS INT) AS fieldGoalsAttempted
,CAST(player_statistics_test_df.fieldGoalsMade AS INT) AS fieldGoalsMade
,player_statistics_test_df.fieldGoalsPercentage
,CAST(player_statistics_test_df.threePointersAttempted AS INT) AS threePointersAttempted
,CAST(player_statistics_test_df.threePointersMade AS INT) AS threePointersMade
,player_statistics_test_df.threePointersPercentage
,CAST(player_statistics_test_df.freeThrowsAttempted AS INT) AS freeThrowsAttempted
,CAST(player_statistics_test_df.freeThrowsMade AS INT) AS freeThrowsMade
,player_statistics_test_df.freeThrowsPercentage
,CAST(player_statistics_test_df.reboundsDefensive AS INT) AS reboundsDefensive
,CAST(player_statistics_test_df.reboundsOffensive AS INT) AS reboundsOffensive
,CAST(player_statistics_test_df.reboundsTotal AS INT) AS reboundsTotal
,CAST(player_statistics_test_df.foulsPersonal AS INT) AS foulsPersonal
,CAST(player_statistics_test_df.turnovers AS INT) AS turnovers
,CAST(player_statistics_test_df.plusMinusPoints AS INT) AS plusMinusPoints
,player_statistics_test_df.season

,All_Star_Flag.all_star_this_season
,MVP_Flag.mvp_this_season
,All_NBA_First_Team_Flag.all_nba_first_team_this_season
,All_NBA_Second_Team_Flag.all_nba_second_team_this_season
,All_NBA_Third_Team_Flag.all_nba_third_team_this_season

FROM player_statistics_test_df
LEFT JOIN All_Star_Flag
ON All_Star_Flag.player_id = player_statistics_test_df.player_id AND All_Star_Flag.Season = player_statistics_test_df.Season
LEFT JOIN MVP_Flag
ON MVP_Flag.player_id = player_statistics_test_df.player_id AND MVP_Flag.Season = player_statistics_test_df.Season
LEFT JOIN All_NBA_First_Team_Flag
ON All_NBA_First_Team_Flag.player_id = player_statistics_test_df.player_id AND All_NBA_First_Team_Flag.Season = player_statistics_test_df.Season
LEFT JOIN All_NBA_Second_Team_Flag
ON All_NBA_Second_Team_Flag.player_id = player_statistics_test_df.player_id AND All_NBA_Second_Team_Flag.Season = player_statistics_test_df.Season
LEFT JOIN All_NBA_Third_Team_Flag
ON All_NBA_Third_Team_Flag.player_id = player_statistics_test_df.player_id AND All_NBA_Third_Team_Flag.Season = player_statistics_test_df.Season


"""

result_df = duckdb.query(query).df()
result_df["all_star_this_season"] = np.where(result_df["all_star_this_season"] == 'Yes',1,0)
result_df["mvp_this_season"] = np.where(result_df["mvp_this_season"] == 'Yes',1,0)
result_df["all_nba_first_team_this_season"] = np.where(result_df["all_nba_first_team_this_season"] == 'Yes',1,0)
result_df["all_nba_second_team_this_season"] = np.where(result_df["all_nba_second_team_this_season"] == 'Yes',1,0)
result_df["all_nba_third_team_this_season"] = np.where(result_df["all_nba_third_team_this_season"] == 'Yes',1,0)
result_df


,firstName,lastName,full_name,player_id,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,...,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints,season,all_star_this_season,mvp_this_season,all_nba_first_team_this_season,all_nba_second_team_this_season,all_nba_third_team_this_season
0,Malcolm,Miller,Malcolm Miller,1626259,21900549,2020-01-07,Toronto,Raptors,Portland,Trail Blazers,...,0,0,0,0,2019,0,0,0,0,0
1,Levi,Randolph,Levi Randolph,1626260,21900548,2020-01-07,Cleveland,Cavaliers,Detroit,Pistons,...,0,0,0,0,2019,0,0,0,0,0
2,Thon,Maker,Thon Maker,1627748,21900548,2020-01-07,Detroit,Pistons,Cleveland,Cavaliers,...,2,0,0,7,2019,0,0,0,0,0
3,Patrick,McCaw,Patrick McCaw,1627775,21900549,2020-01-07,Toronto,Raptors,Portland,Trail Blazers,...,4,2,1,-1,2019,0,0,0,0,0
4,Ante,Zizic,Ante Zizic,1627790,21900548,2020-01-07,Cleveland,Cavaliers,Detroit,Pistons,...,9,1,1,15,2019,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635040,LaMarcus,Aldridge,LaMarcus Aldridge,200746,21001034,2011-03-19,Portland,Trail Blazers,Philadelphia,76ers,...,7,0,1,15,2010,0,0,0,0,1
1635041,Zach,Randolph,Zach Randolph,2216,21001031,2011-03-19,Memphis,Grizzlies,Indiana,Pacers,...,10,2,4,6,2010,0,0,0,0,1
1635042,LaMarcus,Aldridge,LaMarcus Aldridge,200746,21001016,2011-03-17,Portland,Trail Blazers,Cleveland,Cavaliers,...,11,1,2,37,2010,0,0,0,0,1
1635043,Zach,Randolph,Zach Randolph,2216,21001015,2011-03-17,Memphis,Grizzlies,New York,Knicks,...,6,2,2,-8,2010,0,0,0,0,1
